# Import Data from movebank.org

In this case we downloaded data on 43 wolves from Movebank.org that we will use to analyze for a science project.  In our case we studied seasonal variations in wolve movement.  

Before studying the data, we created the scripts below to help understand the data.  We used it to answer

In [2]:
import pandas as pd
import folium

df = pd.read_csv('wolves.csv')
df.head()

/var/folders/vy/5trn1pl97cb26ytv3ypqd8840000gn/T/ipykernel_75163/3964664412.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('wolves.csv')


,event-id,visible,timestamp,location-long,location-lat,external-temperature,gps:dop,height-raw,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name
0,9710294924,True,2013-12-19 00:00:44.000,-111.990073,57.212889,-22.0,2.6,350.59,gps,Canis lupus,13791,13791,Boutin Alberta Grey Wolf
1,9710294925,True,2013-12-19 12:00:45.000,-112.043078,57.211016,-8.0,1.6,335.05,gps,Canis lupus,13791,13791,Boutin Alberta Grey Wolf
2,9710294926,True,2013-12-20 00:00:44.000,-112.105655,57.260090,-5.0,1.8,409.08,gps,Canis lupus,13791,13791,Boutin Alberta Grey Wolf
3,9710294927,True,2013-12-20 12:00:44.000,-112.105336,57.261037,-12.0,1.6,414.98,gps,Canis lupus,13791,13791,Boutin Alberta Grey Wolf
4,9710294928,True,2013-12-21 00:00:44.000,-112.105630,57.260091,-9.0,1.8,416.13,gps,Canis lupus,13791,13791,Boutin Alberta Grey Wolf


In [3]:
df['tag-local-identifier'].unique()

array([13791, 13790, 13793, 13792, 13794, 33677, 15009, 15008, 33670,
       32268, 32269, 33669, 33668, 32260, 32262, 32264, 32265, 32266,
       32267, 35260, 33667, 33681, 33680, 33671, 32251, 33679, 32253,
       32252, 32255, 32254, 32257, 32256, 32259, 32258, 33672, 33673,
       33674, 33675, 33676, 32270, 33678, 32261, 32263])

In [1]:
df['tag-local-identifier'].nunique()

NameError: name 'df' is not defined

In [4]:
wolf1 = df[df['tag-local-identifier'] == 13790]
wolf1.shape

(5531, 13)

# Create basic folium Visualization

We next create a basic visualization with the Folium package.  

In [ ]:
import folium
m = folium.Map(location=[57.212889,-111.990073], zoom_start=12, tiles="Stamen Terrain")

# add marker one by one on the map
for i in range(0,len(wolf1)):
   folium.Circle(
      location=[wolf1.iloc[i]['location-lat'], wolf1.iloc[i]['location-long']],
      popup=wolf1.iloc[i]['timestamp'],
   ).add_to(m)
# Show the map again
m

#  Color telemetry points by wolf

The next visualization will give each wolf a unique color and color the points with this color.  

This visualization allows us to analyze the range and overlap of each wolf's habitat.  

Note that this function will save the result to an html file in the working directory. 

In [ ]:
number_of_colors = 42

colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

# colors = ['#C7980A', '#F4651F', '#82D8A7', '#CC3A05', '#575E76', '#156943', '#0BD055', '#ACD338']

wolves = df['tag-local-identifier'].unique()[:41].tolist()

m = folium.Map(location=[57.212889,-111.990073], zoom_start=12, tiles="Stamen Terrain")

for wolf,color in zip(wolves, colors):
    wolf1 = df[df['tag-local-identifier'] == wolf]
    for i in range(0,len(wolf1)):
       folium.Circle(
          location=[wolf1.iloc[i]['location-lat'], wolf1.iloc[i]['location-long']],
          popup=wolf1.iloc[i]['timestamp'],
            color = color
       ).add_to(m)

# Show the map again
m.save('all_points.html')
# m

# Clustering Points

Sometimes the points can become too crowded.  Clustering points can help with this.  

The following function clusters the points and once again save it to an html file in the working directory. 

In [ ]:
from folium.plugins import MarkerCluster
number_of_colors = 42

colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

# colors = ['#C7980A', '#F4651F', '#82D8A7', '#CC3A05', '#575E76', '#156943', '#0BD055', '#ACD338']

wolves = df['tag-local-identifier'].unique()[:41].tolist()

m = folium.Map(location=[57.212889,-111.990073], zoom_start=12, tiles="Stamen Terrain")
marker_cluster = MarkerCluster().add_to(m)

for wolf,color in zip(wolves, colors):
    wolf1 = df[df['tag-local-identifier'] == wolf]
    for i in range(0,len(wolf1)):
       folium.Circle(
          location=[wolf1.iloc[i]['location-lat'], wolf1.iloc[i]['location-long']],
          popup=wolf1.iloc[i]['timestamp'],
            color = color
       ).add_to(marker_cluster)

# Show the map again
m.save('all_points_clustered.html')
# m

#  Geospatial animation of telemetry points

This is our main contribution.  This code creates TimestampedGeoJson that allows us to play the data through time.  

This code will draw the animal tracks with a line segment between them.  

In order not to crowd the display, we've set the data to only stay on the display for 1 Day

The resulting animation is once again saved to the working directory.  

In [2]:
from folium.plugins import TimestampedGeoJson
import folium
import pandas as pd
import random

df = pd.read_csv('wolves.csv')

number_of_colors = 42

colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]


# colors = ['#C7980A', '#F4651F', '#82D8A7', '#CC3A05', '#575E76', '#156943', '#0BD055', '#ACD338']

wolves = df['tag-local-identifier'].unique().tolist()
# df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.rename(columns = {'location-long':'lon', 'location-lat':'lat'})

m = folium.Map(location=[57.212889,-111.990073], zoom_start=12, tiles="Stamen Terrain")

lines = []
for count,wolf in enumerate(wolves):
    temp = df[df['tag-local-identifier'] == wolf].sort_values(by='timestamp')
    for i, row in temp.iterrows():
        if i == 0:
            continue
        else:
            lines.append(
                {
                    # 'coordinates': [
                    #     [df.iloc[i-1]['lat'], df.iloc[i-1]['lon']],
                    #     [row['lat'], row['lon']],
                    # ],
                     'coordinates': [
                        [ df.iloc[i-1]['lon'],df.iloc[i-1]['lat']],
                        [ row['lon'], row['lat']],
                    ],
                    'dates': [
                        df.iloc[i-1]['timestamp'],
                        df.iloc[i]['timestamp']
                    ],
                    'color': colors[count-1],
                    'popup': df.iloc[i]['tag-local-identifier']
                })

/var/folders/vy/5trn1pl97cb26ytv3ypqd8840000gn/T/ipykernel_83957/195985195.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('wolves.csv')


In [4]:
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
            'popup': str(line['popup']),
            'style': {
                'color': line['color'],
                'weight': 5
            }
        }
    }
    for line in lines
]

TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1H', add_last_point=True, duration = 'P1D', transition_time = 200).add_to(m)

m.save('wolf_movement_animation.html')
# m